In [89]:
import pandas as pd
import numpy as np

# get different data
temp_attr_grid = pd.read_csv("data/grid5_tas_adj_attribution_categories_years.csv")
prec_attr_grid = pd.read_csv("data/grid25_precip_attribution_categories_years.csv")

temp_obs_grid = pd.read_csv("data/grid5_temp_years.csv")
precip_obs_grid = pd.read_csv("data/grid25_prec_years.csv")

grid_region = pd.read_csv('data/subgrid.csv')
grid_region["LON_5"] = grid_region.LON_5.apply(lambda x: x+360 if x<0 else x)
grid_region["LON_25"] = grid_region.LON_25.apply(lambda x: x+360 if x<0 else x)

gdlcodes = pd.read_excel('data/SHDI-SGDI-Total 5.0 2022-05-25.xlsx')
gdlcodes = gdlcodes[['iso_code', 'country', 'GDLCODE',]].drop_duplicates()
gdlcodes = gdlcodes.rename(columns={'GDLCODE':'GDLcode'})

grid_region = grid_region.merge(gdlcodes, on="GDLcode")

original_data = pd.read_csv("data/climate_impact_data_countries.csv")

/home/sarah/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [95]:
# extend original data, since it only contains one faek year
original_data = original_data[['Name', 'ISO', 'year', 'n_studies', 'n_studies_1k_km']]
all_data = []
for year in range(1999,2020):
    one_year = original_data.copy()
    one_year.year = year
    all_data.append(one_year)
original_data = pd.concat(all_data)

In [97]:
def weighted_avg_with_nan(data, weights):
    if np.isnan(data).all():
        return np.nan
    masked = np.ma.masked_array(data, np.isnan(data))
    select = np.invert(np.ma.getmaskarray(masked))
    data = data[select]
    weights = weights[select]
    return np.average(data, weights=weights)

def values_per_region(data, grid, observable, observation, grid_region):
    lon = f"LON_{grid}"
    lat = f"LAT_{grid}"
    
    data = data.merge(grid_region[[lon, lat, "iso_code","country", "area"]], how="left", on=[lon,lat])
         
    if observation:
        data_region = data.groupby(["iso_code","country","year"])\
              .apply(lambda x: weighted_avg_with_nan(data=x[observable], weights=x.area))\
              .reset_index().rename(columns={0:f'obs_{observable}'})
        return data_region[["iso_code","country","year",f'obs_{observable}']]
    
    else: # calculate values for attribution
        # different absolute areas
        # total country area
        areaCountry = data.groupby(["iso_code","country"])[["area"]].sum().reset_index()
        # area with available data
        data_grid_notnan = data.loc[~pd.isna(data["Attribution_Category"])]
        areaCountry_notnan = data_grid_notnan.groupby(["iso_code","country","year"])[["area"]].sum().reset_index().rename(columns={'area':f'da_{observable}_nonna'})
   
        # area where changes can be attributed to human influence
        yearCountry_attr = data.loc[(abs(data.Attribution_Category)>=2) & (abs(data.Attribution_Category)<4)] 
        areaCountry_attr = yearCountry_attr.groupby(["iso_code","country","year"])[["area"]].sum().reset_index().rename(columns={'area':f'da_{observable}_total'})
        
        # calculate relative values
        findat = areaCountry_notnan.merge(areaCountry_attr, on = ["iso_code","country","year"],  how="left").merge(areaCountry, on=["iso_code","country"], how="left")
        findat[f'da_{observable}_nonna'] = findat[f'da_{observable}_nonna']/findat.area
        findat[f'da_{observable}_total'] = findat[f'da_{observable}_total']/findat.area
        return findat[["iso_code","country","year",f'da_{observable}_nonna', f'da_{observable}_total']]
    


In [98]:
temp_obs_reg = values_per_region(temp_obs_grid.rename(columns={"temperature":"temp"}), 5, "temp", True, grid_region)
precip_obs_reg = values_per_region(precip_obs_grid.rename(columns={"precipitation":"precip"}), 25, "precip", True, grid_region)

temp_attr_reg = values_per_region(temp_attr_grid, 5, "temp", False, grid_region)
prec_attr_reg = values_per_region(prec_attr_grid, 25, "precip", False, grid_region)

In [99]:
df = original_data[['Name', 'ISO', 'year', 'n_studies', 'n_studies_1k_km']].merge(
    temp_obs_reg[['iso_code', 'year', 'obs_temp']].rename(columns={"iso_code":"ISO"}), how="left", on=['year', 'ISO']
).merge(
    precip_obs_reg[['iso_code', 'year', 'obs_precip']].rename(columns={"iso_code":"ISO"}), how="left", on=['year', 'ISO']
).merge(
    temp_attr_reg[['iso_code','year', 'da_temp_nonna', 'da_temp_total']].rename(columns={"iso_code":"ISO"}), how="left", on=['year', 'ISO']
).merge(
    prec_attr_reg[['iso_code','year', 'da_precip_nonna', 'da_precip_total']].rename(columns={"iso_code":"ISO"}), how="left", on=['year', 'ISO']
)

In [102]:
df.loc[df.ISO=='ALA']

,Name,ISO,year,n_studies,n_studies_1k_km,obs_temp,obs_precip,da_temp_nonna,da_temp_total,da_precip_nonna,da_precip_total
208,208. Åland Islands,ALA,1999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,208. Åland Islands,ALA,1999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
672,208. Åland Islands,ALA,1999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
904,208. Åland Islands,ALA,1999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1136,208. Åland Islands,ALA,1999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
101360,208. Åland Islands,ALA,2019,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101592,208. Åland Islands,ALA,2019,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101824,208. Åland Islands,ALA,2019,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102056,208. Åland Islands,ALA,2019,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df.to_csv('data/climate_impact_data_countries.csv')